In [936]:
#Bus Speeds

In [937]:
import os

os.getcwd()


'/Users/danielbrown/Desktop/mta-ace-buses/notebooks/EDA'

In [938]:
from pathlib import Path

# Project root = two levels up from notebooks/EDA
PROJECT_ROOT = Path.cwd().parents[1]

DATA_RAW = PROJECT_ROOT / "data" / "raw"

DATA_RAW


PosixPath('/Users/danielbrown/Desktop/mta-ace-buses/data/raw')

In [939]:
list(DATA_RAW.iterdir())


[PosixPath('/Users/danielbrown/Desktop/mta-ace-buses/data/raw/speeds_monthly_2015.csv'),
 PosixPath('/Users/danielbrown/Desktop/mta-ace-buses/data/raw/journeyperformance_monthly_2017.csv'),
 PosixPath('/Users/danielbrown/Desktop/mta-ace-buses/data/raw/bus_lane_geometry.csv'),
 PosixPath('/Users/danielbrown/Desktop/mta-ace-buses/data/raw/able_ace_start.csv'),
 PosixPath('/Users/danielbrown/Desktop/mta-ace-buses/data/raw/waitassessment_monthly_2015.csv')]

In [940]:
import pandas as pd

df_speeds = pd.read_csv(DATA_RAW / "speeds_monthly_2015.csv")
df_speeds.head()


,month,borough,day_type,trip_type,route_id,period,total_operating_time,total_mileage,average_speed
0,2015-01-01,Bronx,1,LCL/LTD,BX1,Off-Peak,"8,710,307","62,940,902.4",7.23
1,2015-01-01,Bronx,1,LCL/LTD,BX1,Peak,"4,334,312","30,316,503.6",6.99
2,2015-01-01,Bronx,2,LCL/LTD,BX1,Off-Peak,"2,498,651","18,742,158",7.50
3,2015-01-01,Bronx,2,LCL/LTD,BX1,Peak,"1,008,139","7,417,580.4",7.36
4,2015-01-01,Bronx,1,LCL/LTD,BX10,Off-Peak,"5,778,595","52,543,814.4",9.09


In [941]:
# Assuming your dataframe is called df_speeds
# Remove commas and convert to numeric
df_speeds["total_operating_time"] = pd.to_numeric(
    df_speeds["total_operating_time"].str.replace(",", ""), errors="coerce"
)
df_speeds["total_mileage"] = pd.to_numeric(
    df_speeds["total_mileage"].str.replace(",", ""), errors="coerce"
)

# Group by month and route_id
df_grouped = (
    df_speeds.groupby(["month", "route_id"], as_index=False)
        .agg({
            "total_mileage": "sum",
            "total_operating_time": "sum"
        })
)

# Calculate average speed per group
df_grouped["average_speed"] = df_grouped["total_mileage"] / df_grouped["total_operating_time"]

# Reset index (optional, just to be safe)
df_grouped.reset_index(drop=True, inplace=True)

# Show result
df_grouped.head()


,month,route_id,total_mileage,total_operating_time,average_speed
0,2015-01-01,B1,134831203.2,17343453.0,7.774184
1,2015-01-01,B100,65153044.8,6604154.0,9.865464
2,2015-01-01,B103,160917087.6,17885202.0,8.997219
3,2015-01-01,B11,89143142.4,14241253.0,6.259501
4,2015-01-01,B12,91564315.2,15104847.0,6.061916


In [942]:
import pandas as pd

df_ace = pd.read_csv(DATA_RAW / "able_ace_start.csv")
df_ace.head()


,Route,Program,Implementation Date
0,M15+,ABLE,10/07/2019
1,B44+,ABLE,10/30/2019
2,M14+,ABLE,11/21/2019
3,B46+,ABLE,02/19/2020
4,M23+,ABLE,08/10/2020


In [943]:
import pandas as pd
from datetime import datetime

df_ace_filtered = df_ace

# Convert Implementation Date to datetime
df_ace_filtered["Implementation Date"] = pd.to_datetime(
    df_ace_filtered["Implementation Date"], format="%m/%d/%Y"
)

# Function to snap to closest first of month
def snap_to_nearest_first_of_month(dt: pd.Timestamp) -> pd.Timestamp:
    first_prev = pd.Timestamp(year=dt.year, month=dt.month, day=1)
    # first of next month
    if dt.month == 12:
        first_next = pd.Timestamp(year=dt.year + 1, month=1, day=1)
    else:
        first_next = pd.Timestamp(year=dt.year, month=dt.month + 1, day=1)
    
    # return whichever is closer
    if (dt - first_prev) <= (first_next - dt):
        return first_prev
    else:
        return first_next

# Apply snapping
df_ace_filtered["Implementation FirstOfMonth"] = df_ace_filtered["Implementation Date"].apply(snap_to_nearest_first_of_month)

# Show result
df_ace_filtered.head()

,Route,Program,Implementation Date,Implementation FirstOfMonth
0,M15+,ABLE,2019-10-07,2019-10-01
1,B44+,ABLE,2019-10-30,2019-11-01
2,M14+,ABLE,2019-11-21,2019-12-01
3,B46+,ABLE,2020-02-19,2020-03-01
4,M23+,ABLE,2020-08-10,2020-08-01


In [944]:
df_grouped["month"] = pd.to_datetime(df_grouped["month"])
df_ace_filtered["Implementation FirstOfMonth"] = pd.to_datetime(
    df_ace_filtered["Implementation FirstOfMonth"]
)


In [945]:
df_ace_filtered = df_ace_filtered.rename(columns={"Route": "route_id"})


In [946]:
program_timeline = (
    df_ace_filtered
    .pivot_table(
        index="route_id",
        columns="Program",
        values="Implementation FirstOfMonth",
        aggfunc="min"
    )
    .reset_index()
)

program_timeline.head()


Program,route_id,ABLE,ACE
0,B11,NaT,2025-11-01
1,B25,2022-12-01,2024-10-01
2,B26,2023-10-01,2024-10-01
3,B35,NaT,2024-09-01
4,B41,NaT,2024-09-01


In [947]:
df = df_grouped.merge(
    program_timeline,
    on="route_id",
    how="inner"
)
df.head()

,month,route_id,total_mileage,total_operating_time,average_speed,ABLE,ACE
0,2015-01-01,B11,89143142.4,14241253.0,6.259501,NaT,2025-11-01
1,2015-01-01,B25,94309812.0,14408075.0,6.545622,2022-12-01,2024-10-01
2,2015-01-01,B26,107962236.0,15654125.0,6.896728,2023-10-01,2024-10-01
3,2015-01-01,B35,233463934.8,36833291.0,6.338395,NaT,2024-09-01
4,2015-01-01,B41,280574413.2,38472276.0,7.292899,NaT,2024-09-01


In [948]:
# Rename columns to Prophet's required schema
df = df.rename(columns={
    "month": "ds",
    "average_speed": "y"
})

# Ensure ds is datetime
df["ds"] = pd.to_datetime(df["ds"])
df

,ds,route_id,total_mileage,total_operating_time,y,ABLE,ACE
0,2015-01-01,B11,8.914314e+07,1.424125e+07,6.259501,NaT,2025-11-01
1,2015-01-01,B25,9.430981e+07,1.440808e+07,6.545622,2022-12-01,2024-10-01
2,2015-01-01,B26,1.079622e+08,1.565412e+07,6.896728,2023-10-01,2024-10-01
3,2015-01-01,B35,2.334639e+08,3.683329e+07,6.338395,NaT,2024-09-01
4,2015-01-01,B41,2.805744e+08,3.847228e+07,7.292899,NaT,2024-09-01
...,...,...,...,...,...,...,...
6698,2025-12-01,Q58,6.563016e+04,8.918476e+03,7.358899,2023-07-01,2024-07-01
6699,2025-12-01,Q6,4.188061e+04,5.416360e+03,7.732243,NaT,2025-09-01
6700,2025-12-01,Q69,2.951430e+04,4.029373e+03,7.324786,NaT,2024-10-01
6701,2025-12-01,S46,4.255627e+04,3.895808e+03,10.923606,NaT,2024-09-01


In [949]:
from prophet import Prophet
import pandas as pd
import numpy as np

# ------------------------------
# Step 1: Select a single route
# ------------------------------
route = "BX19"
df_r = df[df["route_id"] == route].sort_values("ds").copy()

# ------------------------------
# Step 2: Prepare columns for Prophet
# ------------------------------
# Prophet expects 'ds' for datetime and 'y' for target
# We'll add regressors:
# - is_able: 1 if ABLE is active that month
# - is_ace: 1 if ACE is active that month
# - is_covid: 1 if month is during COVID unusual period (e.g., Mar 2020 - Jun 2021)
df_r["is_able"] = (df_r["ABLE"].notna() & (df_r["ds"] >= df_r["ABLE"])).astype(int)
df_r["is_ace"] = (df_r["ACE"].notna() & (df_r["ds"] >= df_r["ACE"])).astype(int)
# Example COVID period — adjust if needed
df_r["is_covid"] = ((df_r["ds"] >= "2020-03-01") & (df_r["ds"] <= "2020-09-01")).astype(int)

# ------------------------------
# Step 3: Fit Prophet model
# ------------------------------
# We'll model pre-policy trends, COVID effects, seasonality, and regressors
m0 = Prophet(
    yearly_seasonality=True,  # bus speeds vary by month
    weekly_seasonality=False, # monthly data, weekly not needed
    daily_seasonality=False
)

# Add regressors
m0.add_regressor("is_able")
m0.add_regressor("is_ace")
m0.add_regressor("is_covid")

# Fit model on historical data
m0.fit(df_r[["ds", "y", "is_able", "is_ace", "is_covid"]])

# ------------------------------
# Step 4: Make predictions WITHOUT policies
# ------------------------------
# This gives us the counterfactual (what speeds would have been without ABLE or ACE)
# Set ABLE and ACE to 0, keep COVID regressor as-is
future = df_r[["ds", "is_covid"]].copy()
future["is_able"] = 0
future["is_ace"] = 0

# Predict
forecast = m0.predict(future)

# Merge predictions back into df_r
df_r = df_r.merge(
    forecast[["ds", "yhat"]],
    on="ds",
    how="left"
)
df_r.rename(columns={"yhat": "yhat_no_policy"}, inplace=True)

# ------------------------------
# Step 5: Compute effects
# ------------------------------
# ABLE effect: average speed increase during ABLE months
able_mask = df_r["is_able"] == 1
ace_mask = df_r["is_ace"] == 1

# ABLE effect: % change vs counterfactual
able_effect = (
    (df_r.loc[able_mask & ~ace_mask, "y"].mean() - 
     df_r.loc[able_mask & ~ace_mask, "yhat_no_policy"].mean())
    / df_r.loc[able_mask & ~ace_mask, "yhat_no_policy"].mean()
) * 100

# ACE incremental effect: % change vs counterfactual (already with ABLE)
ace_effect = (
    (df_r.loc[ace_mask, "y"].mean() - 
     df_r.loc[ace_mask, "yhat_no_policy"].mean())
    / df_r.loc[ace_mask, "yhat_no_policy"].mean()
) * 100

# ------------------------------
# Step 6: Check results
# ------------------------------
print(f"ABLE effect: {able_effect:.2f}%")
print(f"ACE incremental effect: {ace_effect:.2f}%")

# Optional: see the dataframe
df_r[["ds", "y", "yhat_no_policy", "is_able", "is_ace", "is_covid"]].head(20)
df_r

11:59:00 - cmdstanpy - INFO - Chain [1] start processing
11:59:01 - cmdstanpy - INFO - Chain [1] done processing


ABLE effect: -0.32%
ACE incremental effect: 0.43%


,ds,route_id,total_mileage,total_operating_time,y,ABLE,ACE,is_able,is_ace,is_covid,yhat_no_policy
0,2015-01-01,BX19,1.106509e+08,1.988410e+07,5.564794,2022-12-01,2024-07-01,0,0,0,5.566932
1,2015-02-01,BX19,9.992593e+07,1.810596e+07,5.518952,2022-12-01,2024-07-01,0,0,0,5.505386
2,2015-03-01,BX19,1.091200e+08,1.987321e+07,5.490809,2022-12-01,2024-07-01,0,0,0,5.553373
3,2015-04-01,BX19,1.086747e+08,1.987723e+07,5.467297,2022-12-01,2024-07-01,0,0,0,5.467848
4,2015-05-01,BX19,1.103109e+08,2.029015e+07,5.436673,2022-12-01,2024-07-01,0,0,0,5.326433
...,...,...,...,...,...,...,...,...,...,...,...
127,2025-08-01,BX19,4.488492e+04,7.671380e+03,5.850957,2022-12-01,2024-07-01,1,1,0,5.752067
128,2025-09-01,BX19,4.620352e+04,7.907328e+03,5.843127,2022-12-01,2024-07-01,1,1,0,5.742742
129,2025-10-01,BX19,4.920272e+04,8.440022e+03,5.829691,2022-12-01,2024-07-01,1,1,0,5.801621
130,2025-11-01,BX19,4.595002e+04,7.799657e+03,5.891288,2022-12-01,2024-07-01,1,1,0,5.832196


In [950]:
from prophet import Prophet
import pandas as pd
import numpy as np

# ------------------------------
# Step 1: Select a single route
# ------------------------------
route = "BX19"
df_r = df[df["route_id"] == route].sort_values("ds").copy()

# ------------------------------
# Step 2: Prepare columns for Prophet
# ------------------------------
# Prophet requires 'ds' for datetime and 'y' for target variable
# We'll create binary regressors:
# - is_able: 1 if ABLE is active that month
# - is_ace: 1 if ACE is active that month
# - is_covid: 1 if month is during unusual COVID period (bus speeds anomalous)
df_r["is_able"] = (df_r["ABLE"].notna() & (df_r["ds"] >= df_r["ABLE"])).astype(int)
df_r["is_ace"] = (df_r["ACE"].notna() & (df_r["ds"] >= df_r["ACE"])).astype(int)
df_r["is_covid"] = ((df_r["ds"] >= "2020-03-01") & (df_r["ds"] <= "2020-09-01")).astype(int)

# ------------------------------
# Step 3: Fit Prophet model
# ------------------------------
# Model the trend in bus speeds with seasonality, COVID, ABLE, ACE
m0 = Prophet(
    yearly_seasonality=True,  # capture seasonal monthly patterns
    weekly_seasonality=False, # monthly data, weekly not needed
    daily_seasonality=False
)
# Add the policy regressors and COVID regressor
m0.add_regressor("is_able")
m0.add_regressor("is_ace")
m0.add_regressor("is_covid")

# Fit the model to the historical data
m0.fit(df_r[["ds", "y", "is_able", "is_ace", "is_covid"]])

# ------------------------------
# Step 4: Create counterfactual predictions
# ------------------------------
# Counterfactual = what would have happened without ABLE or ACE
future = df_r[["ds", "is_covid"]].copy()  # keep COVID effect as-is
future["is_able"] = 0                       # turn off ABLE
future["is_ace"] = 0                        # turn off ACE

# Generate predictions
forecast = m0.predict(future)

# Merge back into df_r
df_r = df_r.merge(forecast[["ds", "yhat"]], on="ds", how="left")
df_r.rename(columns={"yhat": "yhat_no_policy"}, inplace=True)

# ------------------------------
# Step 5: Define masks for effects
# ------------------------------
# Mask for months post-ABLE but before ACE (ABLE effect only)
able_only_mask = (df_r["is_able"] == 1) & (df_r["is_ace"] == 0)

# Mask for months post-ACE (ACE incremental effect)
ace_mask = df_r["is_ace"] == 1

# ------------------------------
# Step 6: Compute effects
# ------------------------------
# ABLE effect: compare post-ABLE months to pre-ABLE baseline (counterfactual)
able_effect = (
    (df_r.loc[able_only_mask, "y"].mean() - df_r.loc[able_only_mask, "yhat_no_policy"].mean())
    / df_r.loc[able_only_mask, "yhat_no_policy"].mean()
) * 100

# ACE incremental effect: compare post-ACE months to baseline including ABLE (if existed)
ace_effect = (
    (df_r.loc[ace_mask, "y"].mean() - df_r.loc[ace_mask, "yhat_no_policy"].mean())
    / df_r.loc[ace_mask, "yhat_no_policy"].mean()
) * 100

# ------------------------------
# Step 7: Check results
# ------------------------------
print(f"ABLE effect: {able_effect:.2f}%")   # % change due to ABLE
print(f"ACE incremental effect: {ace_effect:.2f}%")  # % change due to ACE on top of ABLE

# Optional: Inspect the dataframe
df_r[["ds", "y", "yhat_no_policy", "is_able", "is_ace", "is_covid", "ABLE", "ACE"]].tail(50)


11:59:01 - cmdstanpy - INFO - Chain [1] start processing
11:59:01 - cmdstanpy - INFO - Chain [1] done processing


ABLE effect: -0.32%
ACE incremental effect: 0.43%


,ds,y,yhat_no_policy,is_able,is_ace,is_covid,ABLE,ACE
82,2021-11-01,5.745058,5.809580,0,0,0,2022-12-01,2024-07-01
83,2021-12-01,5.851027,5.886659,0,0,0,2022-12-01,2024-07-01
84,2022-01-01,6.012720,5.950544,0,0,0,2022-12-01,2024-07-01
85,2022-02-01,5.846285,5.887748,0,0,0,2022-12-01,2024-07-01
86,2022-03-01,5.748840,5.869724,0,0,0,2022-12-01,2024-07-01
87,2022-04-01,5.767972,5.856179,0,0,0,2022-12-01,2024-07-01
88,2022-05-01,5.682400,5.742695,0,0,0,2022-12-01,2024-07-01
89,2022-06-01,5.670467,5.724494,0,0,0,2022-12-01,2024-07-01
90,2022-07-01,5.854461,5.774722,0,0,0,2022-12-01,2024-07-01
91,2022-08-01,5.817770,5.760451,0,0,0,2022-12-01,2024-07-01


In [951]:
'''
from prophet import Prophet
import pandas as pd
import numpy as np

# ------------------------------
# Step 0: Prepare an empty list to collect results
# ------------------------------
results = []

# ------------------------------
# Step 1: Loop over all routes
# ------------------------------
for route in df["route_id"].unique():
    # Select data for this route
    df_r = df[df["route_id"] == route].sort_values("ds").copy()

    # Skip route if it has too few data points
    if len(df_r) < 12:
        continue

    # ------------------------------
    # Step 2: Prepare regressors
    # ------------------------------
    df_r["is_able"] = (df_r["ABLE"].notna() & (df_r["ds"] >= df_r["ABLE"])).astype(int)
    df_r["is_ace"] = (df_r["ACE"].notna() & (df_r["ds"] >= df_r["ACE"])).astype(int)
    df_r["is_covid"] = ((df_r["ds"] >= "2020-03-01") & (df_r["ds"] <= "2020-09-01")).astype(int)

    # ------------------------------
    # Step 3: Fit Prophet model
    # ------------------------------
    try:
        m0 = Prophet(yearly_seasonality=True, weekly_seasonality=False, daily_seasonality=False)
        m0.add_regressor("is_able")
        m0.add_regressor("is_ace")
        m0.add_regressor("is_covid")
        m0.fit(df_r[["ds", "y", "is_able", "is_ace", "is_covid"]])
    except Exception as e:
        print(f"Skipping route {route} due to error in Prophet: {e}")
        continue

    # ------------------------------
    # Step 4: Predict counterfactual (no ABLE or ACE)
    # ------------------------------
    future = df_r[["ds", "is_covid"]].copy()
    future["is_able"] = 0
    future["is_ace"] = 0
    forecast = m0.predict(future)
    df_r = df_r.merge(forecast[["ds", "yhat"]], on="ds", how="left")
    df_r.rename(columns={"yhat": "yhat_no_policy"}, inplace=True)

    # ------------------------------
    # Step 5: Compute effects
    # ------------------------------
    able_only_mask = (df_r["is_able"] == 1) & (df_r["is_ace"] == 0)
    ace_mask = df_r["is_ace"] == 1

    able_effect = (
        (df_r.loc[able_only_mask, "y"].mean() - df_r.loc[able_only_mask, "yhat_no_policy"].mean())
        / df_r.loc[able_only_mask, "yhat_no_policy"].mean()
    ) * 100 if able_only_mask.any() else np.nan  # NaN if no ABLE

    ace_effect = (
        (df_r.loc[ace_mask, "y"].mean() - df_r.loc[ace_mask, "yhat_no_policy"].mean())
        / df_r.loc[ace_mask, "yhat_no_policy"].mean()
    ) * 100 if ace_mask.any() else np.nan  # NaN if no ACE

    # ------------------------------
    # Step 6: Store results
    # ------------------------------
    results.append({
        "route_id": route,
        "able_effect_pct": able_effect,
        "ace_incremental_effect_pct": ace_effect
    })

# ------------------------------
# Step 7: Convert results to dataframe
# ------------------------------


df_effects = pd.DataFrame(results)
df_effects
'''

'\nfrom prophet import Prophet\nimport pandas as pd\nimport numpy as np\n\n# ------------------------------\n# Step 0: Prepare an empty list to collect results\n# ------------------------------\nresults = []\n\n# ------------------------------\n# Step 1: Loop over all routes\n# ------------------------------\nfor route in df["route_id"].unique():\n    # Select data for this route\n    df_r = df[df["route_id"] == route].sort_values("ds").copy()\n\n    # Skip route if it has too few data points\n    if len(df_r) < 12:\n        continue\n\n    # ------------------------------\n    # Step 2: Prepare regressors\n    # ------------------------------\n    df_r["is_able"] = (df_r["ABLE"].notna() & (df_r["ds"] >= df_r["ABLE"])).astype(int)\n    df_r["is_ace"] = (df_r["ACE"].notna() & (df_r["ds"] >= df_r["ACE"])).astype(int)\n    df_r["is_covid"] = ((df_r["ds"] >= "2020-03-01") & (df_r["ds"] <= "2020-09-01")).astype(int)\n\n    # ------------------------------\n    # Step 3: Fit Prophet model\n 

In [952]:
'''
#This is graph 1. It will be a histogram for total_effect_pct
df_effects["total_effect_pct"] = (
    (1 + df_effects["able_effect_pct"].fillna(0) / 100)
    * (1 + df_effects["ace_incremental_effect_pct"].fillna(0) / 100)
    - 1
) * 100

df_effects
'''

'\n#This is graph 1. It will be a histogram for total_effect_pct\ndf_effects["total_effect_pct"] = (\n    (1 + df_effects["able_effect_pct"].fillna(0) / 100)\n    * (1 + df_effects["ace_incremental_effect_pct"].fillna(0) / 100)\n    - 1\n) * 100\n\ndf_effects\n'

In [953]:
'''
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Data
x = df_effects["total_effect_pct"].dropna()

# Choose bin width (policy-friendly)
BIN_WIDTH = 2  # percent points

# Compute bounds
min_val = np.floor(x.min() / BIN_WIDTH) * BIN_WIDTH
max_val = np.ceil(x.max() / BIN_WIDTH) * BIN_WIDTH

# Build bins that include zero explicitly
negative_bins = np.arange(min_val, 0, BIN_WIDTH)
positive_bins = np.arange(0, max_val + BIN_WIDTH, BIN_WIDTH)

bins = np.concatenate([negative_bins, positive_bins])

plt.figure(figsize=(8, 4))

sns.histplot(
    x,
    bins=bins
)

plt.axvline(0, linestyle="--", linewidth=1)
plt.xlabel("Total Effect (% change)")
plt.ylabel("Number of routes")
plt.title("Distribution of Total Program Effects")

plt.tight_layout()
plt.show()
'''

'\nimport numpy as np\nimport seaborn as sns\nimport matplotlib.pyplot as plt\n\n# Data\nx = df_effects["total_effect_pct"].dropna()\n\n# Choose bin width (policy-friendly)\nBIN_WIDTH = 2  # percent points\n\n# Compute bounds\nmin_val = np.floor(x.min() / BIN_WIDTH) * BIN_WIDTH\nmax_val = np.ceil(x.max() / BIN_WIDTH) * BIN_WIDTH\n\n# Build bins that include zero explicitly\nnegative_bins = np.arange(min_val, 0, BIN_WIDTH)\npositive_bins = np.arange(0, max_val + BIN_WIDTH, BIN_WIDTH)\n\nbins = np.concatenate([negative_bins, positive_bins])\n\nplt.figure(figsize=(8, 4))\n\nsns.histplot(\n    x,\n    bins=bins\n)\n\nplt.axvline(0, linestyle="--", linewidth=1)\nplt.xlabel("Total Effect (% change)")\nplt.ylabel("Number of routes")\nplt.title("Distribution of Total Program Effects")\n\nplt.tight_layout()\nplt.show()\n'

In [954]:
'''
# ------------------------------
# Summary statistics for effects
# ------------------------------

summary = {}

for col in ["able_effect_pct", "ace_incremental_effect_pct","total_effect_pct"]:
    series = df_effects[col].dropna()

    summary[col] = {
        "mean": series.mean(),
        "median": series.median(),
        "q25": series.quantile(0.25),
        "q75": series.quantile(0.75),
        "iqr": series.quantile(0.75) - series.quantile(0.25),
        "n_routes": series.shape[0]
    }

summary_df = pd.DataFrame(summary).T
summary_df
'''

'\n# ------------------------------\n# Summary statistics for effects\n# ------------------------------\n\nsummary = {}\n\nfor col in ["able_effect_pct", "ace_incremental_effect_pct","total_effect_pct"]:\n    series = df_effects[col].dropna()\n\n    summary[col] = {\n        "mean": series.mean(),\n        "median": series.median(),\n        "q25": series.quantile(0.25),\n        "q75": series.quantile(0.75),\n        "iqr": series.quantile(0.75) - series.quantile(0.25),\n        "n_routes": series.shape[0]\n    }\n\nsummary_df = pd.DataFrame(summary).T\nsummary_df\n'

In [955]:
'''
import pandas as pd
import numpy as np

def robust_summary(series):
    s = series.dropna()
    return pd.Series({
        "n_routes": s.shape[0],
        "mean": s.mean(),
        "median": s.median(),
        "trimmed_mean_10pct": s.sort_values().iloc[int(0.1*len(s)) : int(0.9*len(s))].mean()
            if len(s) >= 10 else np.nan,
        "std_dev": s.std(),
        "mad": (s - s.median()).abs().median(),
        "min": s.min(),
        "q25": s.quantile(0.25),
        "q75": s.quantile(0.75),
        "max": s.max(),
        "pct_positive": (s > 0).mean() * 100,
        "pct_negative": (s < 0).mean() * 100,
        "pct_near_zero": (s.abs() < 0.5).mean() * 100
    })
'''

'\nimport pandas as pd\nimport numpy as np\n\ndef robust_summary(series):\n    s = series.dropna()\n    return pd.Series({\n        "n_routes": s.shape[0],\n        "mean": s.mean(),\n        "median": s.median(),\n        "trimmed_mean_10pct": s.sort_values().iloc[int(0.1*len(s)) : int(0.9*len(s))].mean()\n            if len(s) >= 10 else np.nan,\n        "std_dev": s.std(),\n        "mad": (s - s.median()).abs().median(),\n        "min": s.min(),\n        "q25": s.quantile(0.25),\n        "q75": s.quantile(0.75),\n        "max": s.max(),\n        "pct_positive": (s > 0).mean() * 100,\n        "pct_negative": (s < 0).mean() * 100,\n        "pct_near_zero": (s.abs() < 0.5).mean() * 100\n    })\n'

In [956]:
'''
summary_stats = pd.concat(
    {
        "ABLE effect (%)": robust_summary(df_effects["able_effect_pct"]),
        "ACE incremental effect (%)": robust_summary(df_effects["ace_incremental_effect_pct"]),
        "TOTAL compounded effect (%)": robust_summary(df_effects["total_effect_pct"])
    },
    axis=1
)

summary_stats
'''

'\nsummary_stats = pd.concat(\n    {\n        "ABLE effect (%)": robust_summary(df_effects["able_effect_pct"]),\n        "ACE incremental effect (%)": robust_summary(df_effects["ace_incremental_effect_pct"]),\n        "TOTAL compounded effect (%)": robust_summary(df_effects["total_effect_pct"])\n    },\n    axis=1\n)\n\nsummary_stats\n'

In [957]:
'''
df_effects["is_sbs"] = df_effects["route_id"].str.contains("\+")

grouped_summary = df_effects.groupby("is_sbs").apply(
    lambda g: robust_summary(g["total_effect_pct"])
)

grouped_summary
'''

'\ndf_effects["is_sbs"] = df_effects["route_id"].str.contains("\\+")\n\ngrouped_summary = df_effects.groupby("is_sbs").apply(\n    lambda g: robust_summary(g["total_effect_pct"])\n)\n\ngrouped_summary\n'

In [958]:
from prophet import Prophet
import pandas as pd
import numpy as np

# ------------------------------
# CONFIG
# ------------------------------
MIN_OBS = 12

COVID_START = "2020-03-01"
COVID_END   = "2021-03-01"   # conservative: include recovery

# ------------------------------
# Step 0: Prepare results list
# ------------------------------
results = []

# ------------------------------
# Step 1: Loop over routes
# ------------------------------
for route in df["route_id"].unique():

    # Select route data
    df_r = df[df["route_id"] == route].sort_values("ds").copy()

    # ------------------------------
    # Step 1a: REMOVE COVID PERIOD
    # ------------------------------
    df_r = df_r[
        ~((df_r["ds"] >= COVID_START) & (df_r["ds"] <= COVID_END))
    ]

    # Skip if too little data remains
    if len(df_r) < MIN_OBS:
        continue

    # ------------------------------
    # Step 2: Policy indicators
    # ------------------------------
    df_r["is_able"] = (
        df_r["ABLE"].notna() & (df_r["ds"] >= df_r["ABLE"])
    ).astype(int)

    df_r["is_ace"] = (
        df_r["ACE"].notna() & (df_r["ds"] >= df_r["ACE"])
    ).astype(int)

    # ------------------------------
    # Step 3: Fit Prophet
    # ------------------------------
    try:
        m0 = Prophet(
            yearly_seasonality=True,
            weekly_seasonality=False,
            daily_seasonality=False,
            changepoint_prior_scale=0.05  # conservative trend
        )

        m0.add_regressor("is_able")
        m0.add_regressor("is_ace")

        m0.fit(df_r[["ds", "y", "is_able", "is_ace"]])

    except Exception as e:
        print(f"Skipping route {route}: {e}")
        continue

    # ------------------------------
    # Step 4: Counterfactual (no ABLE / ACE)
    # ------------------------------
    future = df_r[["ds"]].copy()
    future["is_able"] = 0
    future["is_ace"] = 0

    forecast = m0.predict(future)

    df_r = df_r.merge(
        forecast[["ds", "yhat"]],
        on="ds",
        how="left"
    )

    df_r.rename(columns={"yhat": "yhat_no_policy"}, inplace=True)

    # ------------------------------
    # Step 5: Effect masks
    # ------------------------------
    able_only_mask = (df_r["is_able"] == 1) & (df_r["is_ace"] == 0)
    ace_mask = (df_r["is_ace"] == 1)

    # ------------------------------
    # Step 6: Compute effects
    # ------------------------------
    able_effect = (
        (df_r.loc[able_only_mask, "y"].mean()
         - df_r.loc[able_only_mask, "yhat_no_policy"].mean())
        / df_r.loc[able_only_mask, "yhat_no_policy"].mean()
    ) * 100 if able_only_mask.any() else np.nan

    ace_effect = (
        (df_r.loc[ace_mask, "y"].mean()
         - df_r.loc[ace_mask, "yhat_no_policy"].mean())
        / df_r.loc[ace_mask, "yhat_no_policy"].mean()
    ) * 100 if ace_mask.any() else np.nan

    # ------------------------------
    # Step 7: Store results
    # ------------------------------
    results.append({
        "route_id": route,
        "able_effect_pct": able_effect,
        "ace_incremental_effect_pct": ace_effect
    })

# ------------------------------
# Step 8: Final dataframe
# ------------------------------
df_effects = pd.DataFrame(results)

# ------------------------------
# Step 9: Total compounded effect
# ------------------------------
df_effects["total_effect_pct"] = (
    (1 + df_effects["able_effect_pct"].fillna(0) / 100)
    * (1 + df_effects["ace_incremental_effect_pct"].fillna(0) / 100)
    - 1
) * 100

DATA_PROCESSED = PROJECT_ROOT / "data" / "processed"

DATA_PROCESSED.mkdir(parents=True, exist_ok=True)

df_effects.to_csv(
    DATA_PROCESSED / "speed_overall.csv",
    index=False
)

df_effects

summary_stats = pd.concat(
    {
        "ABLE effect (%)": robust_summary(df_effects["able_effect_pct"]),
        "ACE incremental effect (%)": robust_summary(df_effects["ace_incremental_effect_pct"]),
        "TOTAL compounded effect (%)": robust_summary(df_effects["total_effect_pct"])
    },
    axis=1
)

summary_stats


11:59:01 - cmdstanpy - INFO - Chain [1] start processing
11:59:01 - cmdstanpy - INFO - Chain [1] done processing
11:59:01 - cmdstanpy - INFO - Chain [1] start processing
11:59:01 - cmdstanpy - INFO - Chain [1] done processing
11:59:01 - cmdstanpy - INFO - Chain [1] start processing
11:59:01 - cmdstanpy - INFO - Chain [1] done processing
11:59:01 - cmdstanpy - INFO - Chain [1] start processing
11:59:01 - cmdstanpy - INFO - Chain [1] done processing
11:59:01 - cmdstanpy - INFO - Chain [1] start processing
11:59:01 - cmdstanpy - INFO - Chain [1] done processing
11:59:02 - cmdstanpy - INFO - Chain [1] start processing
11:59:02 - cmdstanpy - INFO - Chain [1] done processing
11:59:02 - cmdstanpy - INFO - Chain [1] start processing
11:59:02 - cmdstanpy - INFO - Chain [1] done processing
11:59:02 - cmdstanpy - INFO - Chain [1] start processing
11:59:02 - cmdstanpy - INFO - Chain [1] done processing
11:59:02 - cmdstanpy - INFO - Chain [1] start processing
11:59:02 - cmdstanpy - INFO - Chain [1]

,ABLE effect (%),ACE incremental effect (%),TOTAL compounded effect (%)
n_routes,20.000000,53.000000,53.000000
mean,1.276132,0.933257,1.502802
median,-0.114299,0.333738,0.318266
trimmed_mean_10pct,0.514104,0.367011,0.367705
std_dev,4.493084,3.440155,6.390713
mad,1.000983,1.163304,1.111921
min,-4.094157,-4.729990,-8.630495
q25,-0.581919,-0.444627,-0.669772
q75,1.334065,1.630937,1.497042
max,15.478066,17.925703,36.178321


In [959]:
# Converting to TypeScript
df = df_effects.copy()

df = df.rename(columns={
    "route_id": "routeId",
    "able_effect_pct": "ableEffectPct",
    "ace_incremental_effect_pct": "aceIncrementalEffectPct",
    "total_effect_pct": "totalEffectPct"
})

# Convert NaN → None
df = df.where(pd.notnull(df), None)


import math

def to_ts_value(v):
    if v is None:
        return "null"
    if isinstance(v, float) and math.isnan(v):
        return "null"
    return round(float(v), 4)


rows = []

for _, r in df.iterrows():
    rows.append(
        f"""  {{
    routeId: "{r.routeId}",
    ableEffectPct: {to_ts_value(r.ableEffectPct)},
    aceIncrementalEffectPct: {to_ts_value(r.aceIncrementalEffectPct)},
    totalEffectPct: {to_ts_value(r.totalEffectPct)}
  }}"""
    )

# 🔧 FIX: join rows *outside* the f-string
rows_joined = ",\n".join(rows)

ts = f"""
export type SpeedPeakRow = {{
  routeId: string;
  ableEffectPct: number | null;
  aceIncrementalEffectPct: number | null;
  totalEffectPct: number;
}};

export const speedPeak: SpeedPeakRow[] = [
{rows_joined}
];
"""

output_path = (
    "/Users/danielbrown/Desktop/mta-ace-buses/src/data/processed/speedOverall.ts"
)

with open(output_path, "w") as f:
    f.write(ts)

print("✅ speedOverall.ts written")


✅ speedOverall.ts written


In [960]:
'''
from prophet import Prophet
import pandas as pd
import numpy as np

# ------------------------------
# CONFIG
# ------------------------------
MIN_OBS = 12
UNCERTAINTY_SAMPLES = 1000
INTERVAL_WIDTH = 0.80   # 80% credible interval

# ------------------------------
# Helper: summarize effect with uncertainty
# ------------------------------
def summarize_effect(df_sub):
    if df_sub.empty:
        return {
            "effect_pct": np.nan,
            "effect_pct_lower": np.nan,
            "effect_pct_upper": np.nan,
            "prob_positive": np.nan
        }

    y_mean = df_sub["y"].mean()

    yhat_mean = df_sub["yhat_no_policy"].mean()
    yhat_lower = df_sub["yhat_no_policy_upper"].mean()  # conservative
    yhat_upper = df_sub["yhat_no_policy_lower"].mean()  # optimistic

    effect_pct = ((y_mean - yhat_mean) / yhat_mean) * 100
    effect_pct_lower = ((y_mean - yhat_lower) / yhat_lower) * 100
    effect_pct_upper = ((y_mean - yhat_upper) / yhat_upper) * 100

    prob_positive = (df_sub["y"] > df_sub["yhat_no_policy"]).mean()

    return {
        "effect_pct": effect_pct,
        "effect_pct_lower": effect_pct_lower,
        "effect_pct_upper": effect_pct_upper,
        "prob_positive": prob_positive
    }

# ------------------------------
# MAIN LOOP
# ------------------------------
results = []

for route in df["route_id"].unique():

    # ------------------------------
    # Step 1: route data
    # ------------------------------
    df_r = df[df["route_id"] == route].sort_values("ds").copy()

    if len(df_r) < MIN_OBS:
        continue

    # ------------------------------
    # Step 2: regressors
    # ------------------------------
    df_r["is_able"] = (
        df_r["ABLE"].notna() & (df_r["ds"] >= df_r["ABLE"])
    ).astype(int)

    df_r["is_ace"] = (
        df_r["ACE"].notna() & (df_r["ds"] >= df_r["ACE"])
    ).astype(int)

    df_r["is_covid"] = (
        (df_r["ds"] >= "2020-03-01") &
        (df_r["ds"] <= "2020-09-01")
    ).astype(int)

    # ------------------------------
    # Step 3: fit Prophet
    # ------------------------------
    try:
        m = Prophet(
            yearly_seasonality=True,
            weekly_seasonality=False,
            daily_seasonality=False,
            uncertainty_samples=UNCERTAINTY_SAMPLES,
            interval_width=INTERVAL_WIDTH
        )

        m.add_regressor("is_able")
        m.add_regressor("is_ace")
        m.add_regressor("is_covid")

        m.fit(df_r[["ds", "y", "is_able", "is_ace", "is_covid"]])

    except Exception as e:
        print(f"Skipping route {route}: {e}")
        continue

    # ------------------------------
    # Step 4: counterfactual forecast (no ABLE, no ACE)
    # ------------------------------
    future = df_r[["ds", "is_covid"]].copy()
    future["is_able"] = 0
    future["is_ace"] = 0

    forecast = m.predict(future)

    df_r = df_r.merge(
        forecast[["ds", "yhat", "yhat_lower", "yhat_upper"]],
        on="ds",
        how="left"
    )

    df_r.rename(
        columns={
            "yhat": "yhat_no_policy",
            "yhat_lower": "yhat_no_policy_lower",
            "yhat_upper": "yhat_no_policy_upper"
        },
        inplace=True
    )

    # ------------------------------
    # Step 5: effect masks
    # ------------------------------
    able_only_mask = (df_r["is_able"] == 1) & (df_r["is_ace"] == 0)
    ace_mask = (df_r["is_ace"] == 1)

    # ------------------------------
    # Step 6: summarize effects
    # ------------------------------
    able_stats = summarize_effect(df_r.loc[able_only_mask])
    ace_stats = summarize_effect(df_r.loc[ace_mask])

    # ------------------------------
    # Step 7: store results
    # ------------------------------
    results.append({
        "route_id": route,

        "able_effect_pct": able_stats["effect_pct"],
        "able_effect_pct_lower": able_stats["effect_pct_lower"],
        "able_effect_pct_upper": able_stats["effect_pct_upper"],
        "able_prob_positive": able_stats["prob_positive"],

        "ace_incremental_effect_pct": ace_stats["effect_pct"],
        "ace_effect_pct_lower": ace_stats["effect_pct_lower"],
        "ace_effect_pct_upper": ace_stats["effect_pct_upper"],
        "ace_prob_positive": ace_stats["prob_positive"]
    })

# ------------------------------
# Step 8: final dataframe
# ------------------------------
df_effects = pd.DataFrame(results)

df_effects
'''

'\nfrom prophet import Prophet\nimport pandas as pd\nimport numpy as np\n\n# ------------------------------\n# CONFIG\n# ------------------------------\nMIN_OBS = 12\nUNCERTAINTY_SAMPLES = 1000\nINTERVAL_WIDTH = 0.80   # 80% credible interval\n\n# ------------------------------\n# Helper: summarize effect with uncertainty\n# ------------------------------\ndef summarize_effect(df_sub):\n    if df_sub.empty:\n        return {\n            "effect_pct": np.nan,\n            "effect_pct_lower": np.nan,\n            "effect_pct_upper": np.nan,\n            "prob_positive": np.nan\n        }\n\n    y_mean = df_sub["y"].mean()\n\n    yhat_mean = df_sub["yhat_no_policy"].mean()\n    yhat_lower = df_sub["yhat_no_policy_upper"].mean()  # conservative\n    yhat_upper = df_sub["yhat_no_policy_lower"].mean()  # optimistic\n\n    effect_pct = ((y_mean - yhat_mean) / yhat_mean) * 100\n    effect_pct_lower = ((y_mean - yhat_lower) / yhat_lower) * 100\n    effect_pct_upper = ((y_mean - yhat_upper) / y

In [961]:
'''
import pandas as pd
import numpy as np
from scipy.stats import skew, kurtosis

# ------------------------------
# Columns to summarize
# ------------------------------
effect_cols = [
    "able_effect_pct", "able_effect_pct_lower", "able_effect_pct_upper", "able_prob_positive",
    "ace_incremental_effect_pct", "ace_effect_pct_lower", "ace_effect_pct_upper", "ace_prob_positive"
]

# ------------------------------
# Prepare summary dictionary
# ------------------------------
summary_stats = {}

for col in effect_cols:
    data = df_effects[col].dropna()
    n = len(data)
    
    if n == 0:
        summary_stats[col] = {k: np.nan for k in [
            "count","mean","median","std","min","q25","q75","max","iqr","skew","kurtosis","frac_positive","frac_ci_positive"
        ]}
        continue
    
    # Determine if this is an "effect" column (vs probability)
    is_effect = "effect" in col and "prob" not in col
    is_lower_bound = "_lower" in col
    
    summary_stats[col] = {
        "count": n,
        "mean": data.mean(),
        "median": data.median(),
        "std": data.std(),
        "min": data.min(),
        "q25": np.percentile(data, 25),
        "q75": np.percentile(data, 75),
        "iqr": np.percentile(data, 75) - np.percentile(data, 25),
        "max": data.max(),
        "skew": skew(data),
        "kurtosis": kurtosis(data),
        "frac_positive": (data > 0).mean() if is_effect else np.nan,
        "frac_ci_positive": (data > 0).mean() if is_lower_bound else np.nan
    }

# ------------------------------
# Convert to DataFrame for display
# ------------------------------
summary_df = pd.DataFrame(summary_stats).T
summary_df
'''

'\nimport pandas as pd\nimport numpy as np\nfrom scipy.stats import skew, kurtosis\n\n# ------------------------------\n# Columns to summarize\n# ------------------------------\neffect_cols = [\n    "able_effect_pct", "able_effect_pct_lower", "able_effect_pct_upper", "able_prob_positive",\n    "ace_incremental_effect_pct", "ace_effect_pct_lower", "ace_effect_pct_upper", "ace_prob_positive"\n]\n\n# ------------------------------\n# Prepare summary dictionary\n# ------------------------------\nsummary_stats = {}\n\nfor col in effect_cols:\n    data = df_effects[col].dropna()\n    n = len(data)\n    \n    if n == 0:\n        summary_stats[col] = {k: np.nan for k in [\n            "count","mean","median","std","min","q25","q75","max","iqr","skew","kurtosis","frac_positive","frac_ci_positive"\n        ]}\n        continue\n    \n    # Determine if this is an "effect" column (vs probability)\n    is_effect = "effect" in col and "prob" not in col\n    is_lower_bound = "_lower" in col\n    \n 

In [962]:
#This will be used for individual routes, when we look at them
'''
from prophet import Prophet
import pandas as pd
import numpy as np

# ------------------------------
# CONFIG
# ------------------------------
MIN_OBS = 12
UNCERTAINTY_SAMPLES = 1000
INTERVAL_WIDTH = 0.80
EPS = 1e-6

# ------------------------------
# Utility functions
# ------------------------------
def pct_effect(y_obs, y_cf):
    """Safe percent effect calculation."""
    y_cf = max(y_cf, EPS)
    return (y_obs - y_cf) / y_cf * 100


def summarize_effect(df_sub):
    """Summarize average effect with uncertainty."""
    if df_sub.empty:
        return {
            "effect_pct": np.nan,
            "effect_pct_lower": np.nan,
            "effect_pct_upper": np.nan,
            "prob_positive": np.nan
        }

    y_mean = df_sub["y"].mean()

    yhat_mean = df_sub["yhat_no_policy"].mean()
    yhat_cf_high = df_sub["yhat_no_policy_upper"].mean()  # conservative
    yhat_cf_low = df_sub["yhat_no_policy_lower"].mean()   # optimistic

    effect_pct = pct_effect(y_mean, yhat_mean)
    effect_pct_lower = pct_effect(y_mean, yhat_cf_high)
    effect_pct_upper = pct_effect(y_mean, yhat_cf_low)

    prob_positive = (df_sub["y"] > df_sub["yhat_no_policy"]).mean()

    return {
        "effect_pct": effect_pct,
        "effect_pct_lower": effect_pct_lower,
        "effect_pct_upper": effect_pct_upper,
        "prob_positive": prob_positive
    }


def combine_effects(a, b):
    """Combine two percent effects multiplicatively."""
    if np.isnan(a) and np.isnan(b):
        return np.nan
    a = 0 if np.isnan(a) else a
    b = 0 if np.isnan(b) else b
    return ((1 + a / 100) * (1 + b / 100) - 1) * 100


# ------------------------------
# MAIN LOOP
# ------------------------------
results = []

for route in df["route_id"].unique():

    # ------------------------------
    # Step 1: route data
    # ------------------------------
    df_r = df[df["route_id"] == route].sort_values("ds").copy()

    if len(df_r) < MIN_OBS:
        continue

    # ------------------------------
    # Step 2: regressors
    # ------------------------------
    df_r["is_able"] = (
        df_r["ABLE"].notna() & (df_r["ds"] >= df_r["ABLE"])
    ).astype(int)

    df_r["is_ace"] = (
        df_r["ACE"].notna() & (df_r["ds"] >= df_r["ACE"])
    ).astype(int)

    df_r["is_covid"] = (
        (df_r["ds"] >= "2020-03-01") &
        (df_r["ds"] <= "2020-09-01")
    ).astype(int)

    # ------------------------------
    # Step 3: fit Prophet
    # ------------------------------
    try:
        m = Prophet(
            yearly_seasonality=True,
            weekly_seasonality=False,
            daily_seasonality=False,
            uncertainty_samples=UNCERTAINTY_SAMPLES,
            interval_width=INTERVAL_WIDTH
        )

        m.add_regressor("is_able")
        m.add_regressor("is_ace")
        m.add_regressor("is_covid")

        m.fit(df_r[["ds", "y", "is_able", "is_ace", "is_covid"]])

    except Exception as e:
        print(f"Skipping route {route}: {e}")
        continue

    # ------------------------------
    # Step 4: counterfactual forecast (no ABLE, no ACE)
    # ------------------------------
    future = df_r[["ds", "is_covid"]].copy()
    future["is_able"] = 0
    future["is_ace"] = 0

    forecast = m.predict(future)

    df_r = df_r.merge(
        forecast[["ds", "yhat", "yhat_lower", "yhat_upper"]],
        on="ds",
        how="left"
    )

    df_r.rename(
        columns={
            "yhat": "yhat_no_policy",
            "yhat_lower": "yhat_no_policy_lower",
            "yhat_upper": "yhat_no_policy_upper"
        },
        inplace=True
    )

    # ------------------------------
    # Step 5: effect masks
    # ------------------------------
    able_only_mask = (df_r["is_able"] == 1) & (df_r["is_ace"] == 0)
    ace_mask = (df_r["is_ace"] == 1)

    # ------------------------------
    # Step 6: summarize effects
    # ------------------------------
    able_stats = summarize_effect(df_r.loc[able_only_mask])
    ace_stats = summarize_effect(df_r.loc[ace_mask])

    # ------------------------------
    # Step 7: store results
    # ------------------------------
    results.append({
        "route_id": route,

        "able_effect_pct": able_stats["effect_pct"],
        "able_effect_pct_lower": able_stats["effect_pct_lower"],
        "able_effect_pct_upper": able_stats["effect_pct_upper"],
        "able_prob_positive": able_stats["prob_positive"],

        "ace_incremental_effect_pct": ace_stats["effect_pct"],
        "ace_effect_pct_lower": ace_stats["effect_pct_lower"],
        "ace_effect_pct_upper": ace_stats["effect_pct_upper"],
        "ace_prob_positive": ace_stats["prob_positive"],
    })

# ------------------------------
# Step 8: final dataframe
# ------------------------------
df_effects = pd.DataFrame(results)

# ------------------------------
# Step 9: total effects (point + uncertainty)
# ------------------------------
df_effects["total_effect_pct"] = df_effects.apply(
    lambda r: combine_effects(
        r["able_effect_pct"],
        r["ace_incremental_effect_pct"]
    ),
    axis=1
)

df_effects["total_effect_pct_lower"] = df_effects.apply(
    lambda r: combine_effects(
        r["able_effect_pct_lower"],
        r["ace_effect_pct_lower"]
    ),
    axis=1
)

df_effects["total_effect_pct_upper"] = df_effects.apply(
    lambda r: combine_effects(
        r["able_effect_pct_upper"],
        r["ace_effect_pct_upper"]
    ),
    axis=1
)

# Probability total effect is positive (union)
df_effects["total_prob_positive"] = (
    1
    - (1 - df_effects["able_prob_positive"].fillna(0))
    * (1 - df_effects["ace_prob_positive"].fillna(0))
)

df_effects
'''

'\nfrom prophet import Prophet\nimport pandas as pd\nimport numpy as np\n\n# ------------------------------\n# CONFIG\n# ------------------------------\nMIN_OBS = 12\nUNCERTAINTY_SAMPLES = 1000\nINTERVAL_WIDTH = 0.80\nEPS = 1e-6\n\n# ------------------------------\n# Utility functions\n# ------------------------------\ndef pct_effect(y_obs, y_cf):\n    """Safe percent effect calculation."""\n    y_cf = max(y_cf, EPS)\n    return (y_obs - y_cf) / y_cf * 100\n\n\ndef summarize_effect(df_sub):\n    """Summarize average effect with uncertainty."""\n    if df_sub.empty:\n        return {\n            "effect_pct": np.nan,\n            "effect_pct_lower": np.nan,\n            "effect_pct_upper": np.nan,\n            "prob_positive": np.nan\n        }\n\n    y_mean = df_sub["y"].mean()\n\n    yhat_mean = df_sub["yhat_no_policy"].mean()\n    yhat_cf_high = df_sub["yhat_no_policy_upper"].mean()  # conservative\n    yhat_cf_low = df_sub["yhat_no_policy_lower"].mean()   # optimistic\n\n    effect

In [963]:
'''
df_curve = (
    df_effects
    .sort_values("total_prob_positive", ascending=False)
    .reset_index(drop=True)
)

df_curve["cum_share"] = (df_curve.index + 1) / len(df_curve)
'''

'\ndf_curve = (\n    df_effects\n    .sort_values("total_prob_positive", ascending=False)\n    .reset_index(drop=True)\n)\n\ndf_curve["cum_share"] = (df_curve.index + 1) / len(df_curve)\n'

In [964]:
'''
import matplotlib.pyplot as plt

plt.figure(figsize=(7, 5))

plt.plot(
    df_curve["total_prob_positive"],
    df_curve["cum_share"],
    marker="o",
    linewidth=2
)

# Reference lines
plt.axvline(0.8, linestyle="--", color="gray", alpha=0.7)
plt.axhline(0.5, linestyle="--", color="gray", alpha=0.7)

plt.xlim(0, 1)
plt.ylim(0, 1)

plt.xlabel("Confidence threshold (P[effect > 0])")
plt.ylabel("Cumulative share of routes")
plt.title("Cumulative success curve for ABLE + ACE program")

plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()
'''

'\nimport matplotlib.pyplot as plt\n\nplt.figure(figsize=(7, 5))\n\nplt.plot(\n    df_curve["total_prob_positive"],\n    df_curve["cum_share"],\n    marker="o",\n    linewidth=2\n)\n\n# Reference lines\nplt.axvline(0.8, linestyle="--", color="gray", alpha=0.7)\nplt.axhline(0.5, linestyle="--", color="gray", alpha=0.7)\n\nplt.xlim(0, 1)\nplt.ylim(0, 1)\n\nplt.xlabel("Confidence threshold (P[effect > 0])")\nplt.ylabel("Cumulative share of routes")\nplt.title("Cumulative success curve for ABLE + ACE program")\n\nplt.grid(True, alpha=0.3)\nplt.tight_layout()\nplt.show()\n'

In [965]:
'''
import pandas as pd
import matplotlib.pyplot as plt

df_bar = df_effects.copy()
df_bar["outcome"] = pd.cut(
    df_bar["total_prob_positive"],
    bins=[0, 0.2, 0.8, 1.0],
    labels=["Likely negative", "Uncertain", "Likely positive"],
    include_lowest=True
)

share = df_bar["outcome"].value_counts(normalize=True).reindex(["Likely negative", "Uncertain", "Likely positive"])

plt.figure(figsize=(6, 2))
plt.barh(
    ["ABLE + ACE"],
    share["Likely negative"],
    color="#d73027",
    label="Likely negative"
)
plt.barh(
    ["ABLE + ACE"],
    share["Uncertain"],
    left=share["Likely negative"],
    color="#fdae61",
    label="Uncertain"
)
plt.barh(
    ["ABLE + ACE"],
    share["Likely positive"],
    left=share["Likely negative"] + share["Uncertain"],
    color="#1a9850",
    label="Likely positive"
)
plt.xlim(0, 1)
plt.xlabel("Share of routes")
plt.title("Overall program impact across routes")
plt.legend(loc='center left', bbox_to_anchor=(1,0.5))
plt.show()
'''

'\nimport pandas as pd\nimport matplotlib.pyplot as plt\n\ndf_bar = df_effects.copy()\ndf_bar["outcome"] = pd.cut(\n    df_bar["total_prob_positive"],\n    bins=[0, 0.2, 0.8, 1.0],\n    labels=["Likely negative", "Uncertain", "Likely positive"],\n    include_lowest=True\n)\n\nshare = df_bar["outcome"].value_counts(normalize=True).reindex(["Likely negative", "Uncertain", "Likely positive"])\n\nplt.figure(figsize=(6, 2))\nplt.barh(\n    ["ABLE + ACE"],\n    share["Likely negative"],\n    color="#d73027",\n    label="Likely negative"\n)\nplt.barh(\n    ["ABLE + ACE"],\n    share["Uncertain"],\n    left=share["Likely negative"],\n    color="#fdae61",\n    label="Uncertain"\n)\nplt.barh(\n    ["ABLE + ACE"],\n    share["Likely positive"],\n    left=share["Likely negative"] + share["Uncertain"],\n    color="#1a9850",\n    label="Likely positive"\n)\nplt.xlim(0, 1)\nplt.xlabel("Share of routes")\nplt.title("Overall program impact across routes")\nplt.legend(loc=\'center left\', bbox_to_anchor